In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import ndcg_score
import dask.dataframe as dd
from dask.dataframe import from_pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_node= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Erdos_tgbn_2024/data/tgbn-genre_node_labels.csv")
df_node['ts']= pd.to_datetime(df_node['ts'], unit= 's') # timestamp to datetime
df_node['ts'] = pd.to_datetime(df_node['ts'].dt.strftime('%Y-%m-%d')) # dont need time because the timestamp is reset to the beginning of day in order to calculate the frequency vector label
df_node['user_id']=df_node['user_id'].str.slice(5).astype('Int64') #convert user_id to integers
df_node # remember weight is the frequency vector entries, not the actual weight

,ts,user_id,genre,weight
0,2005-02-15,54,chillout,0.015835
1,2005-02-15,54,female vocalist,0.015330
2,2005-02-15,54,downtempo,0.008128
3,2005-02-15,54,electronic,0.072162
4,2005-02-15,54,reggae,0.021465
...,...,...,...,...
2741930,2009-06-12,651,katy perry,0.004864
2741931,2009-06-12,724,bebop,0.041841
2741932,2009-06-12,724,jazz,0.958159
2741933,2009-06-12,802,Britney Spears,0.359504


#Train-Val-Test Split

In [4]:
np.quantile(df_node['ts'].unique(), [0.7,0.85]) #70-15-15 split of the timestamps

array(['2008-02-24T14:23:59.999999993', '2008-10-18T07:11:59.999999996'],
      dtype='datetime64[ns]')

In [5]:
train= df_node.loc[df_node['ts']<datetime(2008,2,25)]
val= df_node.loc[(datetime(2008,2,25)<=df_node['ts']) & (df_node['ts']<=datetime(2008,10,18))]
test= df_node.loc[datetime(2008,10,18)<df_node['ts']]

In [6]:
train

,ts,user_id,genre,weight
0,2005-02-15,54,chillout,0.015835
1,2005-02-15,54,female vocalist,0.015330
2,2005-02-15,54,downtempo,0.008128
3,2005-02-15,54,electronic,0.072162
4,2005-02-15,54,reggae,0.021465
...,...,...,...,...
1638965,2008-02-24,995,pop,0.017661
1638966,2008-02-24,995,acoustic,0.146607
1638967,2008-02-24,995,soul,0.088105
1638968,2008-02-24,995,jazz,0.050220


# Some data preprocessing

In [7]:
set(df_node['genre'])-set(train['genre'])

{'jazmine sullivan'}

We will augment an auxillary row for processing

In [8]:
row = {'ts':datetime(2008,2,24), 'user_id': 995, 'genre':'jazmine sullivan','weight':0.0}
train = train._append(row, ignore_index = True)
train

,ts,user_id,genre,weight
0,2005-02-15,54,chillout,0.015835
1,2005-02-15,54,female vocalist,0.015330
2,2005-02-15,54,downtempo,0.008128
3,2005-02-15,54,electronic,0.072162
4,2005-02-15,54,reggae,0.021465
...,...,...,...,...
1638966,2008-02-24,995,acoustic,0.146607
1638967,2008-02-24,995,soul,0.088105
1638968,2008-02-24,995,jazz,0.050220
1638969,2008-02-24,995,female vocalist,0.044934


In [9]:
df_node['genre'].nunique()==train['genre'].nunique()

True

In [10]:
train['user_id'].nunique()

787

# Model :  Rolling average with window size k
We take the average of the last k labels where k is the window size

Firstly we want to know what are the possible window sizes

In [11]:
df_node.groupby(['user_id','genre']).size().describe()

count    122560.000000
mean         22.372185
std          57.450838
min           1.000000
25%           1.000000
50%           5.000000
75%          17.000000
max        1201.000000
dtype: float64

Note that there are entries with only one user genre pair and many as 1201 user genre entries. The average number of entries for a user-genre pair is 22.372185

# Model 4: Window size k=7

In [12]:
train_labels= train.groupby(['user_id','genre']).apply(lambda x: x.tail(7).mean()).reset_index() #note tail=7
train_labels = train_labels.drop('ts', axis=1)
train_labels.rename(columns={'weight':'weight_7'}, inplace=True)
train_labels # we will use this to make prediction

,user_id,genre,weight_7
0,1,80s,0.271609
1,1,Coldplay,0.055873
2,1,Drum and bass,0.094912
3,1,Grunge,0.136364
4,1,Lo-Fi,0.069268
...,...,...,...
86661,1000,soul,0.200897
86662,1000,swing,0.005391
86663,1000,trip hop,0.019362
86664,1000,turntablism,0.011806


In [13]:
train_labels[train_labels.isnull().any(axis=1)] # There are no null entries

,user_id,genre,weight_7


In [14]:
# testing if the above code is working. checking the first entry
df_node[(df_node['user_id']==1) & (df_node['genre']=='80s')].tail(7)

,ts,user_id,genre,weight
676454,2006-11-20,1,80s,0.333333
678575,2006-11-21,1,80s,0.333333
680554,2006-11-22,1,80s,0.102574
692137,2006-11-28,1,80s,0.354545
702754,2006-12-03,1,80s,0.301158
704915,2006-12-04,1,80s,0.076322
719704,2006-12-11,1,80s,0.400000


In [15]:
print(df_node[(df_node['user_id']==1) & (df_node['genre']=='80s')].tail(7)['weight'].mean())
print( "Note that this is the first weight entry in the train_labels column")

0.2716094756428934
Note that this is the first weight entry in the train_labels column


Note that not all users have weight for all 513 genres. We need to fill the missing ones with 0.0.

In [16]:
train_labels= train_labels.groupby(["user_id",'genre'])[["weight_7"]].first().unstack(fill_value=0.0).stack().reset_index()
train_labels # note 787*513=431433

,user_id,genre,weight_7
0,1,00s,0.0
1,1,1970s,0.0
2,1,1980s,0.0
3,1,1990s,0.0
4,1,2000s,0.0
...,...,...,...
403726,1000,westlife,0.0
403727,1000,whitney houston,0.0
403728,1000,world,0.0
403729,1000,wu-tang,0.0


In [17]:
#normalize all the weights. note that for each user they have to sum up to 1
train_labels['weight_7']= train_labels['weight_7'].div(train_labels.groupby(['user_id'])['weight_7'].transform('sum'))

## Inference

Preparing the validation and test sets. We need to make sure that each timestamp, user pair has 513 genre entries. We put 0.0 for entries not in the dataset. 0.0 means no interaction.

In [18]:
val_labels= val.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
val_labels.columns = [*val.columns[:-1], 'weight']
val_labels

,ts,user_id,genre,weight
0,2008-02-25,2,00s,0.0
1,2008-02-25,2,1970s,0.0
2,2008-02-25,2,1980s,0.0
3,2008-02-25,2,1990s,0.0
4,2008-02-25,2,2000s,0.0
...,...,...,...,...
26127598,2008-10-18,1000,westlife,0.0
26127599,2008-10-18,1000,whitney houston,0.0
26127600,2008-10-18,1000,world,0.0
26127601,2008-10-18,1000,wu-tang,0.0


In [19]:
test_labels= test.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
test_labels.columns = [*test.columns[:-1], 'weight']
test_labels

,ts,user_id,genre,weight
0,2008-10-19,1,00s,0.0
1,2008-10-19,1,1970s,0.0
2,2008-10-19,1,1980s,0.0
3,2008-10-19,1,1990s,0.0
4,2008-10-19,1,2000s,0.0
...,...,...,...,...
24818935,2009-06-12,802,westlife,0.0
24818936,2009-06-12,802,whitney houston,0.0
24818937,2009-06-12,802,world,0.0
24818938,2009-06-12,802,wu-tang,0.0


In [20]:
# need to use dask for merging. pandas cannot handle large datasets with current colab memory.
train_labels_dask = from_pandas(train_labels, npartitions=10)
val_labels_dask = from_pandas(val_labels, npartitions=10)
test_labels_dask = from_pandas(test_labels, npartitions=10)

In [21]:
# left join val_labels and train_labels on user_id, genre pair
df_val= dd.merge(val_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])
df_test = dd.merge(test_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])

In [22]:
# from dask back to pandas
df_pred_val= df_val.compute()
df_pred_val

,ts,user_id,genre,weight,weight_7
0,2008-09-04,712,proto-punk,0.0,0.000000
1,2008-09-04,712,psychedelic,0.0,0.053306
2,2008-09-04,712,queer as folk,0.0,0.000000
3,2008-09-04,712,radiohead,0.0,0.000000
4,2008-09-04,712,rap,0.0,0.000000
...,...,...,...,...,...
2601813,2008-10-18,1000,trumpet,0.0,0.000000
2601814,2008-10-18,1000,upbeat,0.0,0.000000
2601815,2008-10-18,1000,vocal,0.0,0.000000
2601816,2008-10-18,1000,whitney houston,0.0,0.000000


In [23]:
# from dask back to pandas
df_pred_test = df_test.compute()
df_pred_test

,ts,user_id,genre,weight,weight_7
0,2009-03-26,178,00s,0.0,0.001631
1,2009-03-26,178,1990s,0.0,0.000000
2,2009-03-26,178,2008,0.0,0.000000
3,2009-03-26,178,80s,0.0,0.015114
4,2009-03-26,178,Alt-country,0.0,0.000000
...,...,...,...,...,...
2479579,2009-06-12,802,prda,0.0,0.000000
2479580,2009-06-12,802,psychobilly,0.0,0.007296
2479581,2009-06-12,802,sexy,0.0,0.001258
2479582,2009-06-12,802,swedish,0.0,0.010755


In [24]:
df_pred_val['weight_7'] = df_pred_val['weight_7'].fillna(0.0) # fill null values with 0.0
df_pred_test['weight_7'] = df_pred_test['weight_7'].fillna(0.0) # fill null values with 0.0

# NDCG Scores

In [25]:
print("The ndcg score on validation set where prediction is the rolling average (window =7) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_val['weight']).reshape(-1,513),np.array(df_pred_val['weight_7']).reshape(-1,513), k=10))

The ndcg score on validation set where prediction is the rolling average (window =7) of observed node labels in training set.
0.22418054241678526


In [26]:
print("The ndcg score on test set where prediction is the rolling average (window =7) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_test['weight']).reshape(-1,513),np.array(df_pred_test['weight_7']).reshape(-1,513), k=10))

The ndcg score on test set where prediction is the rolling average (window =7) of observed node labels in training set.
0.19509420120916246


In [27]:
# Delete the old DataFrames
del train_labels
del val_labels
del test_labels
# Perform garbage collection
del df_pred_val
del df_pred_test

In [28]:
import gc
gc.collect()

0

# Model 5: Window =14

In [12]:
train_labels= train.groupby(['user_id','genre']).apply(lambda x: x.tail(14).mean()).reset_index() #note tail=14
train_labels = train_labels.drop('ts', axis=1)
train_labels.rename(columns={'weight':'weight_14'}, inplace=True)
train_labels # we will use this to make prediction

,user_id,genre,weight_14
0,1,80s,0.292930
1,1,Coldplay,0.055873
2,1,Drum and bass,0.094912
3,1,Grunge,0.136364
4,1,Lo-Fi,0.069268
...,...,...,...
86661,1000,soul,0.200897
86662,1000,swing,0.005391
86663,1000,trip hop,0.019362
86664,1000,turntablism,0.011806


In [13]:
train_labels[train_labels.isnull().any(axis=1)] # There are no null entries

,user_id,genre,weight_14


In [14]:
print(df_node[(df_node['user_id']==1) & (df_node['genre']=='80s')].tail(14)['weight'].mean())
print( "Note that this is the first weight entry in the train_labels column")

0.29292987551570904
Note that this is the first weight entry in the train_labels column


Note that not all users have weight for all 513 genres. We need to fill the missing ones with 0.0.

In [15]:
train_labels= train_labels.groupby(["user_id",'genre'])[["weight_14"]].first().unstack(fill_value=0.0).stack().reset_index()
train_labels # note 787*513=403731

,user_id,genre,weight_14
0,1,00s,0.0
1,1,1970s,0.0
2,1,1980s,0.0
3,1,1990s,0.0
4,1,2000s,0.0
...,...,...,...
403726,1000,westlife,0.0
403727,1000,whitney houston,0.0
403728,1000,world,0.0
403729,1000,wu-tang,0.0


In [16]:
#normalize all the weights. note that for each user they have to sum up to 1
train_labels['weight_14']= train_labels['weight_14'].div(train_labels.groupby(['user_id'])['weight_14'].transform('sum'))

In [17]:
val_labels= val.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
val_labels.columns = [*val.columns[:-1], 'weight']
val_labels

,ts,user_id,genre,weight
0,2008-02-25,2,00s,0.0
1,2008-02-25,2,1970s,0.0
2,2008-02-25,2,1980s,0.0
3,2008-02-25,2,1990s,0.0
4,2008-02-25,2,2000s,0.0
...,...,...,...,...
26127598,2008-10-18,1000,westlife,0.0
26127599,2008-10-18,1000,whitney houston,0.0
26127600,2008-10-18,1000,world,0.0
26127601,2008-10-18,1000,wu-tang,0.0


In [18]:
test_labels= test.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
test_labels.columns = [*test.columns[:-1], 'weight']
test_labels

,ts,user_id,genre,weight
0,2008-10-19,1,00s,0.0
1,2008-10-19,1,1970s,0.0
2,2008-10-19,1,1980s,0.0
3,2008-10-19,1,1990s,0.0
4,2008-10-19,1,2000s,0.0
...,...,...,...,...
24818935,2009-06-12,802,westlife,0.0
24818936,2009-06-12,802,whitney houston,0.0
24818937,2009-06-12,802,world,0.0
24818938,2009-06-12,802,wu-tang,0.0


In [19]:
# need to use dask for merging. pandas cannot handle large datasets with current colab memory.
train_labels_dask = from_pandas(train_labels, npartitions=3)
val_labels_dask = from_pandas(val_labels, npartitions=3)
test_labels_dask = from_pandas(test_labels, npartitions=3)

In [20]:
# left join val_labels and train_labels on user_id, genre pair
df_val= dd.merge(val_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])
df_test = dd.merge(test_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])

In [21]:
# from dask back to pandas
df_pred_val= df_val.compute()
df_pred_val

,ts,user_id,genre,weight,weight_14
0,2008-02-25,2,00s,0.0,0.000000
1,2008-02-25,2,2000s,0.0,0.001865
2,2008-02-25,2,2009,0.0,0.000000
3,2008-02-25,2,3 Doors Down,0.0,0.018710
4,2008-02-25,2,30 seconds to mars,0.0,0.000854
...,...,...,...,...,...
8666011,2008-10-18,1000,viking metal,0.0,0.000000
8666012,2008-10-18,1000,violin,0.0,0.000000
8666013,2008-10-18,1000,vocal jazz,0.0,0.000000
8666014,2008-10-18,1000,westlife,0.0,0.000000


In [22]:
# from dask back to pandas
df_pred_test = df_test.compute()
df_pred_test

,ts,user_id,genre,weight,weight_14
0,2008-10-19,1,00s,0.0,0.0
1,2008-10-19,1,1970s,0.0,0.0
2,2008-10-19,1,1980s,0.0,0.0
3,2008-10-19,1,2000s,0.0,0.0
4,2008-10-19,1,2007,0.0,0.0
...,...,...,...,...,...
8232870,2009-06-12,802,violin,0.0,0.0
8232871,2009-06-12,802,vocal jazz,0.0,0.0
8232872,2009-06-12,802,whitney houston,0.0,0.0
8232873,2009-06-12,802,wu-tang,0.0,0.0


In [23]:
df_pred_val['weight_14'] = df_pred_val['weight_14'].fillna(0.0) # fill null values with 0.0
df_pred_test['weight_14'] = df_pred_test['weight_14'].fillna(0.0) # fill null values with 0.0

# NDCG Scores

In [24]:
print("The ndcg score on validation set where prediction is the rolling average (window =14) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_val['weight']).reshape(-1,513),np.array(df_pred_val['weight_14']).reshape(-1,513), k=10))

The ndcg score on validation set where prediction is the rolling average (window =14) of observed node labels in training set.
0.23331024654079463


In [25]:
print("The ndcg score on test set where prediction is the rolling average (window =14) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_test['weight']).reshape(-1,513),np.array(df_pred_test['weight_14']).reshape(-1,513), k=10))

The ndcg score on test set where prediction is the rolling average (window =14) of observed node labels in training set.
0.2007791278464269


In [26]:
import gc
# Delete the old DataFrames
del train_labels
del val_labels
del test_labels
# Perform garbage collection
del df_pred_val
del df_pred_test
gc.collect()

0

# Model 6: Window =21

In [12]:
train_labels= train.groupby(['user_id','genre']).apply(lambda x: x.tail(21).mean()).reset_index() #note tail=21
train_labels = train_labels.drop('ts', axis=1)
train_labels.rename(columns={'weight':'weight_21'}, inplace=True)
train_labels # we will use this to make prediction

,user_id,genre,weight_21
0,1,80s,0.292930
1,1,Coldplay,0.055873
2,1,Drum and bass,0.094912
3,1,Grunge,0.136364
4,1,Lo-Fi,0.069268
...,...,...,...
86661,1000,soul,0.200897
86662,1000,swing,0.005391
86663,1000,trip hop,0.019362
86664,1000,turntablism,0.011806


In [13]:
#normalize all the weights. note that for each user they have to sum up to 1
train_labels['weight_21']= train_labels['weight_21'].div(train_labels.groupby(['user_id'])['weight_21'].transform('sum'))

In [14]:
train_labels[train_labels.isnull().any(axis=1)] # There are no null entries

,user_id,genre,weight_21


In [15]:
print(df_node[(df_node['user_id']==1) & (df_node['genre']=='80s')].tail(21)['weight'].mean())
print( "Note that this is the first weight entry in the train_labels column")

0.29292987551570904
Note that this is the first weight entry in the train_labels column


Note that not all users have weight for all 513 genres. We need to fill the missing ones with 0.0.

In [16]:
train_labels= train_labels.groupby(["user_id",'genre'])[["weight_21"]].first().unstack(fill_value=0.0).stack().reset_index()
val_labels= val.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
val_labels.columns = [*val.columns[:-1], 'weight']
test_labels= test.groupby(["ts", "user_id",'genre'])["weight"].first().unstack(fill_value=0).stack().reset_index()
test_labels.columns = [*test.columns[:-1], 'weight']

In [17]:
# need to use dask for merging. pandas cannot handle large datasets with current colab memory.
train_labels_dask = from_pandas(train_labels, npartitions=3)
val_labels_dask = from_pandas(val_labels, npartitions=3)
test_labels_dask = from_pandas(test_labels, npartitions=3)

In [18]:
# left join val_labels and train_labels on user_id, genre pair
df_val= dd.merge(val_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])
df_test = dd.merge(test_labels_dask, train_labels_dask, how='left', on=['user_id','genre'])
df_pred_val= df_val.compute()
df_pred_test = df_test.compute()

In [19]:
df_pred_val['weight_21'] = df_pred_val['weight_21'].fillna(0.0) # fill null values with 0.0
df_pred_test['weight_21'] = df_pred_test['weight_21'].fillna(0.0) # fill null values with 0.0

# NDCG Scores

In [20]:
print("The ndcg score on validation set where prediction is the rolling average (window =21) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_val['weight']).reshape(-1,513),np.array(df_pred_val['weight_21']).reshape(-1,513), k=10))

The ndcg score on validation set where prediction is the rolling average (window =21) of observed node labels in training set.
0.23453995189818888


In [21]:
print("The ndcg score on test set where prediction is the rolling average (window =21) of observed node labels in training set.")
print(ndcg_score(np.array(df_pred_test['weight']).reshape(-1,513),np.array(df_pred_test['weight_21']).reshape(-1,513), k=10))

The ndcg score on test set where prediction is the rolling average (window =21) of observed node labels in training set.
0.20144007568178915
